In [4]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

from langchain_community.vectorstores import Chroma
import numpy as np
from typing import List

In [5]:
sample_docs=["""The solar system is a collection of celestial bodies bound together by the gravitational pull of the Sun. At its center lies the Sun, a massive star that provides the light and heat necessary for life on Earth. Surrounding it are eight planets, each with unique characteristics. The inner planets—Mercury, Venus, Earth, and Mars—are rocky and relatively small. The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and composed mostly of gas and ice.
In addition to the planets, the solar system contains moons, asteroids, comets, and dwarf planets like Pluto. The asteroid belt between Mars and Jupiter is home to thousands of rocky bodies, while the Kuiper Belt and Oort Cloud contain icy objects at the edges of the system. Scientists study the solar system to understand not only the origins of Earth but also the possibilities of life beyond our planet."""]
print(sample_docs)

['The solar system is a collection of celestial bodies bound together by the gravitational pull of the Sun. At its center lies the Sun, a massive star that provides the light and heat necessary for life on Earth. Surrounding it are eight planets, each with unique characteristics. The inner planets—Mercury, Venus, Earth, and Mars—are rocky and relatively small. The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and composed mostly of gas and ice.\nIn addition to the planets, the solar system contains moons, asteroids, comets, and dwarf planets like Pluto. The asteroid belt between Mars and Jupiter is home to thousands of rocky bodies, while the Kuiper Belt and Oort Cloud contain icy objects at the edges of the system. Scientists study the solar system to understand not only the origins of Earth but also the possibilities of life beyond our planet.']


In [6]:
import os
import tempfile

# Create a temporary directory (already works in your code)
temp_dir = tempfile.mkdtemp()

# Ensure local ./data/text_files folder exists
os.makedirs("./data/text_files", exist_ok=True)

# Now write files safely
for i, doc in enumerate(sample_docs):
    with open(f"./data/text_files/doc_{i}.txt", "w", encoding="utf-8") as f:
        f.write(doc)

print(f"Sample documents created in: {os.path.abspath('./data/text_files')}")


Sample documents created in: /Users/user/Documents/RAGUDEMY/2-vectorstore/data/text_files


In [7]:
import os
print("Current working dir:", os.getcwd())


Current working dir: /Users/user/Documents/RAGUDEMY/2-vectorstore


In [8]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    path="/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/",
    glob="*.txt",
    loader_cls=TextLoader,   # <--- force it to use TextLoader
    loader_kwargs={"encoding": "utf-8"}
)

docs = loader.load()
print(f"Loaded {len(docs)} documents")
print(docs[0])
docs


Loaded 1 documents
page_content='Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary
This session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live demo of a novel prompt extraction attack (revealing system prompts without model access), and understand why traditional security tools fall short. The talk also covers how attackers exploit generative AI and provides practical defenses to strengthen AI LLM security posture.

Why Attend:

Understand common vulnerabilities in LLM applications.

Learn from Black Hat trainers and pen testers real-world findings.

S

[Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/python.txt'}, page_content="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary\nThis session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live demo of a novel prompt extraction attack (revealing system prompts without model access), and understand why traditional security tools fall short. The talk also covers how attackers exploit generative AI and provides practical defenses to strengthen AI LLM security posture.\n\nWhy Attend:\n\nUnderstand common vulnerabi

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    length_function=len,   # ✅ correct argument
    separators=["\n", " ", ".", "\n\n", "\n\n\n"]  # ✅ name should be separators
)
chunks=text_splitter.split_documents(docs)

print(len(chunks))


3


In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

# init embeddings
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# convert Document objects -> list of strings
texts = [doc.page_content for doc in chunks]

# generate embeddings
vector = embeddings_model.embed_documents(texts)

print(len(vector), "embeddings created")
print("First embedding vector length:", len(vector[0]))
print(vector)


3 embeddings created
First embedding vector length: 384
[[-0.030431121587753296, 0.058468788862228394, -0.052441880106925964, -0.004236826207488775, -0.01571459323167801, -0.11361173540353775, 0.0013876183656975627, 0.06451823562383652, -0.029804306104779243, -0.027357056736946106, -0.1179594099521637, 0.052956949919462204, 0.09355516731739044, -0.010082781314849854, 0.08080901950597763, -0.0072097922675311565, -0.002887786366045475, 0.0020401140209287405, -0.0068238344974815845, -0.05148882418870926, 0.0033004165161401033, 0.06818991154432297, 0.00742182694375515, -0.009852803312242031, -0.0747167095541954, 0.0398201122879982, -2.9867924240534194e-05, 0.0005819569923914969, 0.014980732463300228, -0.024894895032048225, 0.0030398082453757524, 0.10470005869865417, 0.04971923306584358, 0.02541264146566391, 0.027617845684289932, 0.03658105060458183, 0.02491465024650097, -0.08256809413433075, -0.027827631682157516, 0.0027429868932813406, -0.09640645980834961, 0.05136973038315773, -0.0401886

In [14]:
persist_directory = "./chroma_db"

vectorstore=Chroma.from_documents(documents=chunks, embedding=embeddings_model, persist_directory=persist_directory,collection_name="rag")
print(f"Vectorstore created:{vectorstore._collection.count()}")


Vectorstore created:3


In [12]:
from langchain_community.vectorstores import Chroma

persist_directory = "./chroma_db"

# Connect to your persisted DB
db = Chroma(
    persist_directory=persist_directory,
    collection_name="rag",
    embedding_function=embeddings_model
)

# Now delete the collection
db._client.delete_collection("rag")
print("Collection 'rag' deleted ✅")


Collection 'rag' deleted ✅


/var/folders/sw/q0vghxyj0cs6_3glp9q55pq80000gp/T/ipykernel_38192/2904532259.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [15]:
query = "What is the solar system?"

results_scores = vectorstore.similarity_search_with_relevance_scores(query, k=3)

for i, (doc, score) in enumerate(results_scores, 1):
    print(f"Result {i}:")
    print(f"Score: {score:.4f}")
    print(f"Content: {doc.page_content[:200]}...\n")



Result 1:
Score: -0.2087
Content: Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language const...

Result 2:
Score: -0.3557
Content: This session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live ...

Result 3:
Score: -0.4231
Content: Why Attend:

Understand common vulnerabilities in LLM applications.

Learn from Black Hat trainers and pen testers real-world findings.

See live demonstrations of prompt-based attacks.

Gain actionab...



/var/folders/sw/q0vghxyj0cs6_3glp9q55pq80000gp/T/ipykernel_38192/305248199.py:3: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/python.txt'}, page_content="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary"), -0.20873385171617498), (Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/python.txt'}, page_content='This session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live demo of a novel prompt extrac

In [16]:
import os
from langchain_openai import ChatOpenAI

# ⚠️ Replace with your actual OpenAI key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.4)
response = llm.invoke("Hello, how are you?")
print(response)





content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 13, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6YdiiSjgLnuq5qNvl16C21jGXvYa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--e5638dd6-fb0a-44f4-b73a-96108ce588b7-0' usage_metadata={'input_tokens': 13, 'output_tokens': 33, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


Modern rag chain

In [17]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings_model, persist_directory="./chroma_db")


In [18]:
retriever=vectorstore.as_retriever(search_kwargs={"k": 2})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x153437260>, search_kwargs={'k': 2})

In [19]:
from langchain.prompts import ChatPromptTemplate
system_prompt = """
You are a QA assistant that answers questions strictly using the retrieved documents or data provided.

Rules:
1. Answer only based on the retrieved information.
2. If the answer is not found or only partially matches the question, respond clearly: "Out of context."
3. Provide concise answers and quote or reference the relevant part of the retrieved data whenever possible.
4. Do not make assumptions or invent information.
5. Maintain a professional, clear, and friendly tone.

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])
    


In [20]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are a QA assistant that answers questions strictly using the retrieved documents or data provided.\n\nRules:\n1. Answer only based on the retrieved information.\n2. If the answer is not found or only partially matches the question, respond clearly: "Out of context."\n3. Provide concise answers and quote or reference the relevant part of the retrieved data whenever possible.\n4. Do not make assumptions or invent information.\n5. Maintain a professional, clear, and friendly tone.\n\nContext:\n{context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are a QA assistant that answers questions strictly using the retrieved documents or data provided.\n\nRules:\n1. Answer only based on the retrieved information.\n2. If the answer is not found or only partially matches the question, respond clearly: "Out of context."\n3. Provide concise answers and quote or reference the relevant part of the retrieved data whenever possible.\n4. Do not make assumptions or invent information.\n5. Maintain a professional, clear, and friendly tone.\n\nContext:\n{context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTempl

In [22]:
from langchain.chains import create_retrieval_chain
rag_chain=create_retrieval_chain(retriever,document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x153437260>, search_kwargs={'k': 2}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are a QA assistant that answers questions strictly using the retrieved documents or data provided.\n\nRules:\n1. Answer only b

In [23]:
response = rag_chain.invoke({"input": "What is python ?"})
response


{'input': 'What is python ?',
 'context': [Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/python.txt'}, page_content="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary"),
  Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/python.txt'}, page_content="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summar

In [24]:
response['answer']  




'"Python is an interpreted, high-level and general-purpose programming language. Python\'s design philosophy emphasizes code readability with its notable use of significant whitespace."'

In [25]:
vectorstore

LCEL chain

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

In [29]:
custom_prompt= ChatPromptTemplate.from_template("""You are a helpful QA assistant. 

- Answer only using the provided context.  
- If the answer is not in the context, reply: "Out of context."  
- Keep your answers short, clear, and professional.  

Context:
{context} 

Question: {question}
Answer:
""")  
custom_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='You are a helpful QA assistant. \n\n- Answer only using the provided context.  \n- If the answer is not in the context, reply: "Out of context."  \n- Keep your answers short, clear, and professional.  \n\nContext:\n{context} \n\nQuestion: {question}\nAnswer:\n'), additional_kwargs={})])

In [27]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [30]:
rag_chain_lcel=(
    {"context":retriever | format_docs,"question": RunnablePassthrough()}
    |custom_prompt
    |llm
    |StrOutputParser()
)
response=rag_chain_lcel.invoke("What is the python?")
response

'Python is an interpreted, high-level and general-purpose programming language.'

In [31]:
chunks

[Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/python.txt'}, page_content="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary"),
 Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/python.txt'}, page_content='This session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live demo of a novel prompt extraction attack (revealing system prompts without model access), and understand why traditional security tools fall short. The talk also covers how attackers exploit ge

Conversational memory

In [41]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage,AIMessage

In [36]:
contextualize_q_system_prompt="""Given a chat history and latest user question which might reference context in chat history,formulate 
a standalone question which can be understood without the chat history.Do not answer the question,
just reformulate it if needed and otherwise return as it is"""
contextualize_q_prompt=ChatPromptTemplate.from_messages([
    ("system",contextualize_q_system_prompt),MessagesPlaceholder("chat_history"),("human","{input}"),])

In [38]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x309d831a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x337e5ac60>, root_client=<openai.OpenAI object at 0x309b91580>, root_async_client=<openai.AsyncOpenAI object at 0x309d81550>, temperature=0.4, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [42]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x153437260>, search_kwargs={'k': 2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag=

In [43]:
qa_system_prompt="""you are an assistant for question-answering task.
use the folowing pieces of retrieved context to answer the question.
if you dont know the answer,just say that you don't know.Use three sentences maximum and keep the answer concise

Context:{context}"""

qa_prompt=ChatPromptTemplate.from_messages([
    ("system",qa_system_prompt),
    MessagesPlaceholder("chat_history"),("human","{input}"),
])
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)

In [44]:
conversational_rag_chain=create_retrieval_chain(
    history_aware_retriever,
    question_answer_chain
)

In [ ]:
chat_history=[]
result1=conversational_rag_chain.invoke({"chat_history":chat_history,
"input":"what is python?"})

print(result1['answer'])

Python is an interpreted, high-level and general-purpose programming language known for its emphasis on code readability and significant whitespace. It aims to help programmers write clear, logical code for projects of various scales.


In [ ]:
chat_history.extend([HumanMessage(content="what is python"),
AIMessage(content=result1['answer'])])

In [ ]:
result2=conversational_rag_chain.invoke({
    "chat_history":chat_history,
    "input":"what are its main types?"
})
result2

{'chat_history': [HumanMessage(content='what is python', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Python is an interpreted, high-level and general-purpose programming language known for its emphasis on code readability and significant whitespace. It aims to help programmers write clear, logical code for projects of various scales.', additional_kwargs={}, response_metadata={})],
 'input': 'what are its main types?',
 'context': [Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/0-DataIngestParsing/data/text_files/python.txt'}, page_content="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary"),
  Document(metadata={'source': '/Users/user/Documents/RAGUDEMY/

LCEL:groq

In [53]:
from dotenv import load_dotenv
load_dotenv()

import os

from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:gemm2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x3053119a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x305311df0>, model_name='gemm2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
simple_prompt=ChatPromptTemplate.from_template("""Answer the question based only on following context:
context:{context}
question:{question}
Answer:""")